In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re

## 드라이버 설치
https://chromedriver.chromium.org/downloads

#드라이버랑 크롬버젼이랑 일치시켜주어야함. 드라이버 다운 받을 때 참고(메뉴 => 도움말 => Chrome 정보에서 확인)

In [2]:
#Step 1. 크롬 웹브라우저 실행
#경로설정
path = "C:/Users/sojeo/OneDrive/바탕 화면/chromedriver_win32/chromedriver.exe"   # 맥은 "chromedriver"
driver = webdriver.Chrome(path)

In [3]:
# txt_file_path = 'productUrlList.txt'

In [5]:
# #productUrlList.txt에서 링크 리스트 불러오기
# r = open(txt_file_path, mode='rt', encoding='utf-8')
# productUrlList = r.readlines()
# len(productUrlList)

In [7]:
import json

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt']

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    error_ls = []
    prod_data = []
    satisfaction_data = []

    for prod_url in tqdm(productUrlList) :
        try :
            driver.get(prod_url)
            time.sleep(1)

            #상품 정보 뽑기

            #상품 고유번호
            prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
            prod_idx = re.sub('\n','',prod_idx)
            prod_idx = int(prod_idx)

            #상품명
            productTitle= driver.find_element_by_class_name("deal_title_main").text

            #상품 카테고리 최대 6층
            category_ls = []
            for i in range(1, 7) :
                try :
                    category= driver.find_element_by_class_name("category._depth{}".format(i)).text
                    category_ls.append(category)
                except :
                    break

            #상품 가격
            productPrice = driver.find_element_by_class_name("number_unit").text #'21,900원'으로 나오기 때문에 전처리 필요 정규표현식으로 숫자만 남기고 int
            temp = re.findall('[0-9]*', productPrice)
            productPrice = int(''.join(temp))

            #구매횟수 
            buy_count = driver.find_element_by_class_name("deal_price_buy_count").text
            temp = re.findall('[0-9]*', buy_count)
            buy_count = int(''.join(temp))

            #상품 만족도 star_rate_num
            try :
                prod_satis = float(driver.find_element_by_class_name("grade_average_score").text)
            except :
                prod_satis = 0.

            #상품이미지 url
            image = driver.find_element_by_class_name("active").find_element_by_tag_name('img').get_attribute('src')

            #상품종료여부(품절 = 0, 구매가능 = 1)
            if driver.find_elements_by_class_name("col2")[1].text == '바로구매' :
                stocks = 1
            else : 
                stocks = 0


            prod_data.append({
                'prod_idx' : prod_idx,
                'image' : image,
                'title' : productTitle,
                'category' : category_ls,
                'price' : productPrice,
                'stocks' : stocks,
                'prod_satis' : prod_satis,
                'prod_url' : prod_url,
                'buy_count' : buy_count
            })

            #상품 만족도 & 배송 만족도
            customer_satisfaction = driver.find_elements_by_class_name("customer_satisfaction_box")
            avg_seller_prod_satis = int(customer_satisfaction[0].find_element_by_class_name('num').text)
            avg_seller_ship_satis = int(customer_satisfaction[1].find_element_by_class_name('num').text)

            try : #옵션 없는 경우 존재
                review_stats = driver.find_elements_by_class_name("review_stats_item")

                #옵션1과 만족도 (ex: 맛 - 73)
                use_opt1 = review_stats[0].find_element_by_class_name('keyword').text
                avg_use_satis1 = review_stats[0].find_element_by_class_name('pct').text
                avg_use_satis1 = re.sub('\%','',avg_use_satis1)
                avg_use_satis1 = int(avg_use_satis1)

                #옵션2와 만족도
                use_opt2 = review_stats[1].find_element_by_class_name('keyword').text
                avg_use_satis2 = review_stats[1].find_element_by_class_name('pct').text
                avg_use_satis2 = re.sub('\%','',avg_use_satis2)
                avg_use_satis2 = int(avg_use_satis2)

            except :
                use_opt1 =''
                avg_use_satis1 = 0
                use_opt2 =''
                avg_use_satis2 = 0

            satisfaction_data.append({
                'prod_idx' : prod_idx,
                'avg_seller_prod_satis' : avg_seller_prod_satis,
                'avg_seller_ship_satis' : avg_seller_ship_satis,
                'use_opt1' : use_opt1,
                'avg_use_satis1' : avg_use_satis1,
                'use_opt2' : use_opt2,
                'avg_use_satis2' : avg_use_satis2,
                'prod_satis' : prod_satis,
                'prod_url' : prod_url,   
            })

        except : 
            error_ls.append(prod_url)

    toJson(prod_data, 'prod_data_{}.json'.format(간편식_냉장_냉동))
    toJson(satisfaction_data, 'satisfaction_data_{}.json'.format(간편식_냉장_냉동))
    toJson(error_ls, 'error_data_{}.json'.format(간편식_냉장_냉동))

#중간에 멈췄으면, 아래 코드 실행 후 **반복문 범위와 **파일저장명(file_path) 변경해서 재실행
#print(productUrlList.index(prod_data[-1]))

In [11]:
satisfaction_data_df = pd.read_json('satisfaction_data_간편식_냉장_냉동.json')
satisfaction_data_df

,prod_idx,avg_seller_prod_satis,avg_seller_ship_satis,use_opt1,avg_use_satis1,use_opt2,avg_use_satis2,prod_satis,prod_url
0,537157714,93,93,맛,70,포장,66,4.6,http://www.tmon.co.kr/deal/537157714\n
1,311679606,95,95,맛,69,포장,64,4.6,http://www.tmon.co.kr/deal/311679606\n
2,279069445,94,93,맛,72,포장,68,4.6,http://www.tmon.co.kr/deal/279069445\n
3,3223462394,91,92,맛,63,포장,62,4.6,http://www.tmon.co.kr/deal/3223462394\n
4,517007326,93,94,맛,67,포장,63,4.5,http://www.tmon.co.kr/deal/517007326\n
...,...,...,...,...,...,...,...,...,...
11334,4069161210,88,89,포장,58,맛,58,4.4,http://www.tmon.co.kr/deal/4069161210\n
11335,3590960242,88,90,포장,69,맛,68,4.5,http://www.tmon.co.kr/deal/3590960242\n
11336,4298084638,92,91,맛,68,포장,64,4.7,http://www.tmon.co.kr/deal/4298084638\n
11337,4312329630,92,91,맛,68,포장,63,4.6,http://www.tmon.co.kr/deal/4312329630\n


In [9]:
prod_data_df = pd.read_json('prod_data_간편식_냉장_냉동.json')
prod_data_df

,prod_idx,image,title,category,price,stocks,prod_satis,prod_url,buy_count
0,537157714,http://th2.tmon.kr/thumbs/image/a13/77c/f3f/f0...,[굽네] 신제품 출시 닭가슴살/간편식 골라담기+증정,"[식품·건강, 간편식·냉장·냉동, 닭가슴살·오리훈제, 닭가슴살]",1200,1,4.6,http://www.tmon.co.kr/deal/537157714\n,1127051
1,311679606,http://th4.tmon.kr/thumbs/image/36f/327/916/ff...,[랭킹닭컴] 맛있닭/잇메이트/러브잇/베스틱 BEST 닭가슴살 모음전,"[식품·건강, 간편식·냉장·냉동, 닭가슴살·오리훈제, 닭가슴살]",1640,1,4.6,http://www.tmon.co.kr/deal/311679606\n,549234
2,279069445,http://img3.tmon.kr/cdn3/deals/2019/05/08/2790...,[무꼬뭐꼬] 추억의 궁물떡볶이 게릴라 990원 3팩/5팩 구매시 사은품증정,"[식품·건강, 간편식·냉장·냉동, 떡볶이·면·떡·기타, 떡볶이]",5900,1,4.6,http://www.tmon.co.kr/deal/279069445\n,371231
3,3223462394,http://th2.tmon.kr/thumbs/image/cbf/ebd/f0a/1e...,[퍼스트위크] 티몬블랙딜 오쿡 닭가슴살 1+1+1+1 외 33종 2900원 균일가 ...,"[식품·건강, 간편식·냉장·냉동, 닭가슴살·오리훈제, 닭가슴살]",2900,1,4.6,http://www.tmon.co.kr/deal/3223462394\n,368506
4,517007326,http://th2.tmon.kr/thumbs/image/550/d2f/6e5/3c...,"[마이닭] 닭가슴살 BEST 한정할인혜택/\n닭가슴살스테이크,닭가슴살볶음밥,닭가슴살...","[식품·건강, 간편식·냉장·냉동, 소시지·핫도그·피자·어묵, 볼/꼬치/봉어묵]",1450,1,4.5,http://www.tmon.co.kr/deal/517007326\n,229404
...,...,...,...,...,...,...,...,...,...
11740,4069161210,http://th1.tmon.kr/thumbs/image/7d2/6ec/452/4d...,[퍼스트위크] 티몬블랙딜 통통떡볶이1-2인분 외 순대/튀김/어묵/핫바 모음,"[식품·건강, 간편식·냉장·냉동, 떡볶이·면·떡·기타, 떡볶이]",1900,1,4.4,http://www.tmon.co.kr/deal/4069161210\n,23282
11741,3590960242,http://th1.tmon.kr/thumbs/image/fc3/45a/d3d/27...,[퍼스트위크] 티몬블랙딜 자연과농부 흑마늘 무뼈족발 350g + 350g 외 4종,"[식품·건강, 간편식·냉장·냉동, 양념·육가공·수산, 족발]",7900,1,4.5,http://www.tmon.co.kr/deal/3590960242\n,10479
11742,4298084638,http://th1.tmon.kr/thumbs/image/2b4/03d/fa9/33...,[퍼스트위크] 필수특가 아임닭 닭가슴살/소시지/핫바 외 BEST 18종 균일가전,"[식품·건강, 간편식·냉장·냉동, 닭가슴살·오리훈제, 닭가슴살]",2500,1,4.7,http://www.tmon.co.kr/deal/4298084638\n,2943
11743,4312329630,http://th1.tmon.kr/thumbs/image/1be/8b4/b5c/03...,[퍼스트위크] 티몬블랙딜 아임닭 닭가슴살/볶음밥/소시지/간식류 90종 골라담기 / ...,"[식품·건강, 간편식·냉장·냉동, 볶음밥·죽·간편식, 볶음밥]",1900,1,4.6,http://www.tmon.co.kr/deal/4312329630\n,13006


In [12]:
error_data_df = pd.read_json('error_data_간편식_냉장_냉동.json')
error_data_df

,0
0,http://www.tmon.co.kr/deal/487445606\n
1,http://www.tmon.co.kr/deal/266769989\n
2,http://www.tmon.co.kr/deal/988151566\n
3,http://www.tmon.co.kr/deal/261963937\n
4,http://www.tmon.co.kr/deal/429491766\n
...,...
464,http://www.tmon.co.kr/deal/4457412774\n
465,http://www.tmon.co.kr/deal/4456722818\n
466,http://www.tmon.co.kr/deal/4456721726\n
467,http://www.tmon.co.kr/deal/4456721570\n
